In [1]:
import sys
from calldb import CallDB


req_id = 'rq0001'

suitability = CallDB("pro_carrier_suitability_assessment")
request = suitability.from_db(f"""
SELECT * 
FROM pro_carrier_suitability_assessment.request 
WHERE req_id = "{req_id}"
;
""")

drug_name = request.loc[0, 'drug_name']
smiles = request.loc[0, 'smiles']
protein_name = request.loc[0, 'protein_name']
sequence = request.loc[0, 'sequence']
weight = request.loc[0, 'weight']

##################
# 타겟 약물이 친수성인지 소수성인지 확인
from polarity import Polarity
polarity = Polarity(smiles)

In [2]:
##################
# 타겟 약물 인코딩
import numpy as np
import torch
from rdkit import Chem
from rdkit.Chem import AllChem

drug_m = Chem.MolFromSmiles(smiles)
drug_fp = torch.tensor(np.array(AllChem.GetMorganFingerprintAsBitVect(drug_m, 3, nBits=1024)), dtype=torch.float32)

In [3]:
# 전달체 후보군 finger print 불러오기
call_db = CallDB('pro_carrier_suitability_assessment')
pol_matched = call_db.from_db(f"""
SELECT Drugbank_ID, name, finger_print, block_type, block_score, SMILES
FROM pro_carrier_suitability_assessment.block_type_library 
WHERE block_type = {polarity};
""")
pol_matched = pol_matched[['Drugbank_ID', 'name', 'finger_print', 'block_type', 'block_score', 'SMILES']]

carrier_fp = []
for i, (_, _, fp, _, _, _) in pol_matched.iterrows():
    carrier_fp.append(np.fromstring(fp.replace('', ' '), dtype=int, sep=' '))
    
carrier_fp = torch.tensor(carrier_fp, dtype=torch.float32)

In [4]:
import DDI_runner

ddi_pred = DDI_runner.run(drug_fp, carrier_fp)
ddi_pred = list(np.array(ddi_pred.detach()))

/BiO/projects/polarity/carrier_suitability_assessment/DDI_runner.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  combi_data = carrier_fp + torch.tensor(drug_fp, dtype=torch.float32)


In [5]:
# DDI 결과를 바탕으로 BA 계산할 전달체 후보 추출
interaction = CallDB("pro_drug_drug_interaction")
inter_dict = interaction.from_db("""
SELECT *
FROM pro_drug_drug_interaction.interaction_dictionary
;
""")

import pandas as pd
ddi_type = pd.DataFrame(np.array(inter_dict[['label', 'interaction_type']].iloc[ddi_pred]), columns=['label', 'interaction_type'])   # interaction 점수 가져오기
ddi_matched = pol_matched[['Drugbank_ID', 'finger_print', 'SMILES']].iloc[list(ddi_type.index[ddi_type['interaction_type']==2])]   # 만족하는 점수만 선택

carrier_fp = []    # carrier_fp 여기서 다시 초기화
for i, (id, fp, _) in ddi_matched.iterrows():
    carrier_fp.append(np.fromstring(fp.replace('', ' '), dtype=int, sep=' '))
    
carrier_fp = torch.tensor(carrier_fp, dtype=torch.float32)

In [6]:
# 타겟 단백질 인코딩
amino_dict = {s: i for i, s in enumerate(list('ACDEFGHIKLMNOPQRSTVWXY'))}
sequence_embd = [amino_dict[c] for c in sequence.upper()]

max_len = 10000
for i in range(max_len - len(sequence_embd)):
    sequence_embd.append(0)

sequence_embd = torch.tensor(sequence_embd, dtype=torch.float32)[:2048].view(1, 2048)
sequence_temp = torch.tensor([], dtype=torch.float32).view(0, 2048)

for i in range(len(carrier_fp)):
    sequence_temp = torch.cat([sequence_temp, sequence_embd], dim=0)
sequence_embd = sequence_temp

In [7]:
# Binding_affinity 예측
import BA_runner

ba_pred = BA_runner.run(carrier_fp, sequence_embd, len(carrier_fp))
ba_pred = np.round(np.array(ba_pred.detach()), 4)
ba_pred = np.where(ba_pred < 0, 0.0001, ba_pred)

In [8]:
# DB_result에 삽입할 DF 만들기
drugbank_id = pd.DataFrame(ddi_matched['Drugbank_ID'], columns=['Drugbank_ID'])
result = drugbank_id
pol_result = pol_matched

ddi_result1 = ddi_type[ddi_type['interaction_type']==2]

ddi_result2 = inter_dict.iloc[list(ddi_type[ddi_type['interaction_type']==2]['label'])][['label', 'interaction']]
ddi_result2 = pd.DataFrame(np.array(ddi_result2), columns=['label', 'interaction'])

ba_result = pd.DataFrame(ba_pred, columns=['ic50', 'ec50'])

request_id = pd.DataFrame({'req_id' : [req_id for i in range(len(result))]})

In [9]:
result = result.join(ddi_result1).merge(pol_result).join(ba_result).join(ddi_result2[['interaction']]).join(request_id)
result = result[['req_id', 'Drugbank_ID', 'name', 'block_type', 'block_score', 'interaction', 'interaction_type', 'ic50', 'ec50']]
result

,req_id,Drugbank_ID,name,block_type,block_score,interaction,interaction_type,ic50,ec50
0,rq0001,DB00413,Pramipexole,2,1.22560,B may increase the sedative activities of A.,2,153.429092,16.745501
1,rq0001,DB01255,Lisdexamfetamine,2,1.52347,A may decrease the sedative and stimulatory ac...,2,114.651199,10.030200
2,rq0001,DB01299,Sulfadoxine,2,1.32545,The therapeutic efficacy of A can be increased...,2,94.085800,17.893499
3,rq0001,DB04794,Bifonazole,2,1.23374,The therapeutic efficacy of A can be increased...,2,111.761597,14.452000
4,rq0001,DB06636,Isavuconazonium,2,1.03744,The therapeutic efficacy of A can be increased...,2,177.249207,8.805400
5,rq0001,DB07213,(5-{3-[5-(PIPERIDIN-1-YLMETHYL)-1H-INDOL-2-YL]...,2,1.45959,The therapeutic efficacy of A can be increased...,2,51.810600,17.457800
6,rq0001,DB07328,"METHYL 4-{[({[(2R,5S)-5-{[(2S)-2-(AMINOMETHYL)...",2,1.23144,The therapeutic efficacy of B can be increased...,2,107.552299,0.000100
7,rq0001,DB08301,N-({[4-(AMINOSULFONYL)PHENYL]AMINO}CARBONYL)-4...,2,1.15022,The therapeutic efficacy of B can be increased...,2,200.235794,15.132000
8,rq0001,DB13307,Proscillaridin,2,1.26761,B may decrease the cardiotoxic activities of A.,2,0.977700,9.808400
9,rq0001,DB15323,Trelagliptin,2,1.05415,The therapeutic efficacy of A can be increased...,2,65.155602,12.831300


In [10]:
import math

for idx, (req_id, Drugbank_ID, name, block_type, block_score, interaction, interaction_type, ic50, ec50) in result.iterrows():
    interaction = interaction.replace('A', drug_name).replace('B', name)
    
    n_ref_ae = suitability.from_db(f"""
    SELECT COUNT(*)
    FROM pro_carrier_suitability_assessment.abstract_adverse_effect
    WHERE abstract like '%{name}%'
    """
                                  ).iloc[0,0]
                                    
    total_score = (20 * block_score) + (10 * interaction_type) + (-math.log(ic50)) + (-math.log(ec50))
        
    suitability.query_db(f"""
    INSERT INTO pro_carrier_suitability_assessment.result(req_id, 
                                                          idx, 
                                                          Drugbank_ID, 
                                                          name, 
                                                          block_type, 
                                                          block_score, 
                                                          interaction, 
                                                          interaction_type, 
                                                          ic50, ec50, 
                                                          n_ref_da, 
                                                          n_ref_ae,
                                                          total_score)
    VALUES ('{req_id}', 
             {idx}, 
             '{Drugbank_ID}', 
             '{name}', 
             {block_type}, 
             {round(block_score, 3)}, 
             '{interaction}', 
             {interaction_type}, 
             {round(ic50, 3)}, 
             {round(ec50, 3)},
             0, 
             {n_ref_ae},
             {total_score});    
    """)
    
#     print(f"""
#     INSERT INTO pro_carrier_suitability_assessment.result(req_id, idx, Drugbank_ID, name, block_type, block_score, interaction, interaction_type, ic50, ec50)
#     VALUES ('{req_id}', {idx}, '{Drugbank_ID}', '{name}', {block_type}, {round(block_score, 3)}, '{interaction}', {interaction_type}, {round(ic50, 3)}, {round(ec50, 3)});    
#     """)


In [11]:
for idx, (req_id, Drugbank_ID, name) in result[['req_id', 'Drugbank_ID', 'name']].iterrows():
    suitability.query_db(f"""
    INSERT INTO pro_carrier_suitability_assessment.result_adverse_effects_ref (req_id, index_id, reference_title, year, summary)
    (
    SELECT '{req_id}', {idx}, title, year, summary
    FROM pro_carrier_suitability_assessment.abstract_adverse_effect
    WHERE abstract like '%{name}%'
    );
    """)

In [12]:
# 분자 이미지 저장
from rdkit.Chem import Draw
import os

smiles = pd.DataFrame(ddi_matched[['Drugbank_ID', 'SMILES']])
os.system(f'mkdir /BiO/projects/polarity/carrier_suitability_assessment/img_result/{req_id}')
for i, (id, smi) in smiles.iterrows():
    m = Chem.MolFromSmiles(smi)
    Draw.MolToFile(m, f'/BiO/projects/polarity/carrier_suitability_assessment/img_result/{req_id}/{id}.png')